### Stock Analysis

In this section, we will perform analysis using RSI, volatility, and other basic financial tools.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Creating a function to retrieve financial data from yfinance.

In [2]:
def get_stock_data(ticker, period='1y'):
    df = yf.download(ticker, period=period, auto_adjust=True)
    
    # Sütun isimlerindeki karmaşıklığı (MultiIndex) temizle
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    
    # Getirileri Hesapla
    # Log Getiri (Analiz ve Volatilite için daha doğru)
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    return df

Creating a function that generates technical indicators.

In [3]:
def add_indicators(df):
    # Simple Moving Average (SMA 20 and 50)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

Creating a function to generate a candlestick plot.

In [4]:
def volatility_analysis(df):
    volatility = df['Log_Return'].rolling(window=21).std() * np.sqrt(252)  # Annualized Volatility
    df['Volatility'] = volatility
    return df

def volatility_for_period(df):
    yearly_volatility = df['Log_Return'].resample('Y').std() * np.sqrt(252)
    return yearly_volatility

# Volatility Analysis for multiple symbols
symbol1 = "NVDA"
data1 = get_stock_data(symbol1)
data1 = add_indicators(data1)

# Volatility Analysis for symbol1
data1 = volatility_analysis(data1)
print(f"Volatility Analysis for {symbol1}:")
print(data1[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol2 = "AAPL"
data2 = get_stock_data(symbol2)
data2 = add_indicators(data2)

# Volatility Analysis for symbol2
data2 = volatility_analysis(data2)
print(f"Volatility Analysis for {symbol2}:")
print(data2[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol3 = "GARAN.IS"
data3 = get_stock_data(symbol3)
data3 = add_indicators(data3)
# Volatility Analysis for symbol3
data3 = volatility_analysis(data3)
print(f"Volatility Analysis for {symbol3}:")
print(data3[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed



Volatility Analysis for NVDA:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  185.550003    0.017067    0.382635  48.879194
2025-12-09  184.970001   -0.003131    0.382711  54.338761
2025-12-10  183.779999   -0.006454    0.321538  46.384468
2025-12-11  180.929993   -0.015629    0.310405  50.432113
2025-12-12  175.020004   -0.033210    0.326260  45.047403


[*********************100%***********************]  1 of 1 completed



Volatility Analysis for AAPL:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841


[*********************100%***********************]  1 of 1 completed

Volatility Analysis for GARAN.IS:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  141.199997   -0.002122    0.268843  71.428548
2025-12-09  140.600006   -0.004258    0.249783  65.873008
2025-12-10  139.699997   -0.006422    0.239410  59.051707
2025-12-11  139.199997   -0.003586    0.239618  64.018676
2025-12-12  140.399994    0.008584    0.240230  62.801895


Creating a function to generate plot-candlestick

In [5]:
def plot_dashboard(df, ticker):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=(f'{ticker} Price Analysis', 'RSI Momentum'),
                        row_width=[0.2, 0.7])

    # Candlestick Graph
    fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'], name='Fiyat'), row=1, col=1)
    
    # Checking if SMA columns exist before plotting
    if 'SMA_20' in df.columns and 'SMA_50' in df.columns:
    # SMA Lines
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='blue', width=1), name='SMA 20'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], line=dict(color='orange', width=1), name='SMA 50'), row=1, col=1)
        

    # Checking if RSI column exists before plotting
    if 'RSI' in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], line=dict(color='purple', width=2), name='RSI'), row=2, col=1)
        # Reference lines for RSI at 70 and 30
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Checking if Volatility column exists before plotting
    if 'Volatility' in df.columns:    
    # Volatility Line
        fig.add_trace(go.Scatter(x=df.index, y=df['Volatility'] * 100, line=dict(color='green', width=1), name='Volatility'), row=1, col=1)


    # Layout adjustments
    fig.update_layout(title=f'{ticker} Technical Analysis Dashboard', yaxis_title='Price', xaxis_rangeslider_visible=False, height=800)
    fig.show()

## Using the functions:

In [6]:
symbol = "GARAN.IS" # Or "AAPL", "NVDA"

# 1. Get Data
data = get_stock_data(symbol)

# 2. Add Indicators
data = add_indicators(data)
data = volatility_analysis(data)

# 3. Plot Dashboard
plot_dashboard(data, symbol)

# 4. Show Last 5 Days
print(data[['Close', 'Log_Return', 'RSI', 'Volatility']].tail())

[*********************100%***********************]  1 of 1 completed



Price            Close  Log_Return        RSI  Volatility
Date                                                     
2025-12-08  141.199997   -0.002122  71.428548    0.268843
2025-12-09  140.600006   -0.004258  65.873008    0.249783
2025-12-10  139.699997   -0.006422  59.051707    0.239410
2025-12-11  139.199997   -0.003586  64.018676    0.239618
2025-12-12  140.399994    0.008584  62.801895    0.240230


## Package

In [7]:
# Creating packaged analysis function
def analysis(symbol, period='1y'):
    data = get_stock_data(symbol, period=period)
    data = add_indicators(data)
    data = volatility_analysis(data)
    return data

AAPL_data = analysis("AAPL", "10y")
print(AAPL_data[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())
# AAPL_data.to_csv("AAPL_analysis.csv")

plot_dashboard(AAPL_data, "AAPL")

[*********************100%***********************]  1 of 1 completed

Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841


## Expected Returns and Covariance Matrix

In [8]:
def get_portfolio_history(symbols, period='1y'):
    portfolio_df = pd.DataFrame()
    
    for symbol in symbols:
        data = get_stock_data(symbol, period=period) 
        portfolio_df[symbol] = data['Log_Return']
        
    return portfolio_df


symbols = ["AAPL", "NVDA", "GARAN.IS"]


df_portfolio = get_portfolio_history(symbols)

expected_returns = df_portfolio.mean() * 252

covariance_matrix = df_portfolio.cov() * 252

print("--- Beklenen Yıllık Getiriler ---")
print(expected_returns)
print("\n--- Kovaryans Matrisi (Risk İlişkisi) ---")
print(covariance_matrix)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

--- Beklenen Yıllık Getiriler ---
AAPL        0.120624
NVDA        0.268668
GARAN.IS    0.064886
dtype: float64

--- Kovaryans Matrisi (Risk İlişkisi) ---
              AAPL      NVDA  GARAN.IS
AAPL      0.105281  0.067297  0.010511
NVDA      0.067297  0.248530  0.017397
GARAN.IS  0.010511  0.017397  0.149207


## Portfolio Volatility & Return

In [9]:
symbols = ["AAPL", "NVDA", "INTC"]
weights = [0.4, 0.4, 0.2] # Example Weights

def portfolio_performance(symbols, weights, period='1y'):
    df_portfolio = get_portfolio_history(symbols, period=period)
    expected_returns = df_portfolio.mean() * 252
    covariance_matrix = df_portfolio.cov() * 252
    weights = np.array(weights)
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_return, portfolio_volatility

df_portfolio = get_portfolio_history(symbols, period='1y')
def portfolio_performance_with_data(portfolio_df, weights, period='1y'):
    expected_returns = portfolio_df.mean() * 252
    covariance_matrix = portfolio_df.cov() * 252
    weights = np.array(weights)
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_return, portfolio_volatility

portfolio_return, portfolio_volatility = portfolio_performance_with_data(df_portfolio, weights, period='1y')
print(f"\nPortfolio({symbols}) Expected Annual Return: {portfolio_return:.2%}")
print(f"Portfolio({symbols}) Volatility (Risk): {portfolio_volatility:.2%}")

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed


Portfolio(['AAPL', 'NVDA', 'INTC']) Expected Annual Return: 28.12%
Portfolio(['AAPL', 'NVDA', 'INTC']) Volatility (Risk): 34.79%


## Getting best returns from yfinance

In [10]:

# 1. BIST Hisseleri Listesi (Sonuna .IS eklenmiş hali)
# Not: Buraya BIST100'ün tamamını da kopyalayabilirsin.
xu_tickers = [
    "THYAO.IS", "AKBNK.IS", "GARAN.IS", "ASELS.IS", "SISE.IS", 
    "KCHOL.IS", "SAHOL.IS", "TUPRS.IS", "BIMAS.IS", "EREGL.IS", 
    "FROTO.IS", "ISCTR.IS", "YKBNK.IS", "VAKBN.IS", "HALKB.IS",
    "PETKM.IS", "TCELL.IS", "ARCLK.IS", "ENKAI.IS", "TOASO.IS",
    "PGSUS.IS", "KOZAL.IS", "KRDMD.IS", "SASA.IS", "HEKTS.IS",
    "EKGYO.IS", "VESTL.IS", "ASTOR.IS", "REEDR.IS", "MGROS.IS"
]

def get_top_performers(tickers, period="1y"):
    print(f"{len(tickers)} hisse için veri indiriliyor...")
    
    # 2. Toplu Veri İndirme (Sadece 'Close' fiyatları yeterli)
    # threads=True işlemi hızlandırır
    data = yf.download(tickers, period=period, progress=False, auto_adjust=True)['Close']
    
    # 3. Getiri Hesaplama (Total Return)
    # Formül: (Son Fiyat / İlk Fiyat) - 1
    # iloc[0]: Dönem başı fiyatı, iloc[-1]: Dönem sonu (güncel) fiyat
    returns = (data.iloc[-1] / data.iloc[0]) - 1
    
    # 4. Sıralama ve İlk 10'u Seçme
    top_10 = returns.sort_values(ascending=False).head(10)
    
    return top_10, data

# --- UYGULAMA ---
top_gainers, all_data = get_top_performers(xu_tickers, period="1y")

print("-" * 40)
print("🏆 BIST - SON 1 YILIN EN ÇOK KAZANDIRANLARI")
print("-" * 40)

for symbol, ret in top_gainers.items():
    print(f"{symbol.replace('.IS', ''):<10} : %{ret*100:.2f}")



print("-" * 40)

30 hisse için veri indiriliyor...
----------------------------------------
🏆 BIST - SON 1 YILIN EN ÇOK KAZANDIRANLARI
----------------------------------------
ASELS      : %195.17
HALKB      : %119.18
KOZAL      : %65.26
EKGYO      : %59.22
ENKAI      : %58.94
TUPRS      : %44.06
VAKBN      : %28.41
TOASO      : %27.29
YKBNK      : %19.42
GARAN      : %15.73
----------------------------------------
----------------------------------------
🏆 BIST - SON 1 YILIN EN ÇOK KAZANDIRANLARI
----------------------------------------
ASELS      : %195.17
HALKB      : %119.18
KOZAL      : %65.26
EKGYO      : %59.22
ENKAI      : %58.94
TUPRS      : %44.06
VAKBN      : %28.41
TOASO      : %27.29
YKBNK      : %19.42
GARAN      : %15.73
----------------------------------------


In [11]:
# getting top 10 symbols
top_10_symbols = top_gainers.index.tolist()

print("Top 10 Symbols for Further Analysis:", top_10_symbols)

Top 10 Symbols for Further Analysis: ['ASELS.IS', 'HALKB.IS', 'KOZAL.IS', 'EKGYO.IS', 'ENKAI.IS', 'TUPRS.IS', 'VAKBN.IS', 'TOASO.IS', 'YKBNK.IS', 'GARAN.IS']


## Best Portfolio with Monte Carlo Simulation

In [12]:
import plotly.graph_objects as go

symbols = top_10_symbols
period = '1y'

symbols = ["AAPL", "NVDA", "INTC"]
df_portfolio = get_portfolio_history(symbols, period=period)

# ---  Simülasyon Ayarları ---
num_portfolios = 10000 # 10000 senaryo deneyelim
all_weights = np.zeros((num_portfolios, len(symbols)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

print(f"{num_portfolios} senaryo hesaplanıyor... for {period}")

# --- 3. Hızlı Döngü ---
for i in range(num_portfolios):
    # Rastgele ağırlıklar
    weights = np.random.random(len(symbols))
    weights = weights / np.sum(weights)
    all_weights[i, :] = weights
    
    # Performans Hesapla
    ret, vol = portfolio_performance_with_data(df_portfolio, list(weights), period='1y')
    
    ret_arr[i] = ret
    vol_arr[i] = vol
    sharpe_arr[i] = ret / vol

# --- 4. En İyisini Bul ---
max_sharpe_idx = sharpe_arr.argmax()
best_ret = ret_arr[max_sharpe_idx]
best_vol = vol_arr[max_sharpe_idx]
best_w = all_weights[max_sharpe_idx, :]

print("-" * 30)
print(f"EN İYİ PORTFÖY (Max Sharpe: {sharpe_arr[max_sharpe_idx]:.2f})")
print(f"Getiri: %{best_ret:.2%}")
print(f"Risk:   %{best_vol:.2%}")
print("DAĞILIM:")
for i, s in enumerate(symbols):
    print(f"{s}: %{best_w[i]:.2%}")
print("-" * 30)


# --- 5. Görselleştirme ---
fig = go.Figure()

# Tüm Senaryolar
fig.add_trace(go.Scatter(
    x=vol_arr, y=ret_arr,
    mode='markers',
    marker=dict(
        size=5,
        color=sharpe_arr,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Sharpe Ratio")
    ),
    text=[f"Sharpe: {s:.2f}" for s in sharpe_arr],
    name='Olası Portföyler'
))

# En İyi Nokta
fig.add_trace(go.Scatter(
    x=[best_vol], y=[best_ret],
    mode='markers',
    marker=dict(color='red', size=15, symbol='star'),
    name='OPTIMAL PORTFÖY'
))

fig.update_layout(
    title=f'{symbols} Efficient Frontier Analizi',
    xaxis_title='Risk (Volatilite)',
    yaxis_title='Getiri (Return)',
    height=600
)

fig.show()

print("Bitti.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed



10000 senaryo hesaplanıyor... for 1y
------------------------------
EN İYİ PORTFÖY (Max Sharpe: 1.02)
Getiri: %54.40%
Risk:   %53.19%
DAĞILIM:
AAPL: %0.31%
NVDA: %22.81%
INTC: %76.88%
------------------------------
------------------------------
EN İYİ PORTFÖY (Max Sharpe: 1.02)
Getiri: %54.40%
Risk:   %53.19%
DAĞILIM:
AAPL: %0.31%
NVDA: %22.81%
INTC: %76.88%
------------------------------


Bitti.


## Risk Analysis
### VaR (Value at Risk) analysis for one stock

In [13]:
from scipy.stats import norm

def calculate_parametric_var(df, portfolio_value=100000, confidence_level=0.95, day=1):
    """
    Hull'un 'Linear Model'ini (Parametrik VaR) kullanarak riski hesaplar.
    """
    
    current_annual_volatility = df['Volatility'].iloc[-1]
    
    daily_volatility = current_annual_volatility / np.sqrt(252)

    z_score = norm.ppf(confidence_level)

    var_value = portfolio_value * daily_volatility * z_score * np.sqrt(day)
    
    return var_value, current_annual_volatility

size = 525
trust = 0.99
symbol = "TSLA"
day = 1

data = get_stock_data(symbol, period='1y')
data = volatility_analysis(data)  # Add volatility analysis
data = add_indicators(data)
data.to_csv(f"{symbol}_volatility_analysis.csv")
var, annual_volatility = calculate_parametric_var(data, portfolio_value=size, confidence_level=trust, day=day)
print(f"{symbol} için Parametrik VaR ({trust*100:.0f}% güven seviyesi) : ${var:.2f}")
print(f"Yani maksimum {day} günlük kayıp: %{var/size*100:.2f}")
print(f"Yıllık Volatilite: %{annual_volatility*100:.2f}")

[*********************100%***********************]  1 of 1 completed

TSLA için Parametrik VaR (99% güven seviyesi) : $33.09
Yani maksimum 1 günlük kayıp: %6.30
Yıllık Volatilite: %43.01


## Historical VaR

In [14]:
def calculate_historical_var(df, portfolio_value=100000, confidence_level=0.95, day=1):
    """
    Tarihsel Simülasyon yöntemi ile VaR hesaplar.
    """

    # 1. Eğer vade 1 günden fazlaysa, geçmişteki o vadeli getirileri oluştur
    if day == 1:
        # 1 günse olduğu gibi al
        period_returns = df['Log_Return'].dropna()
    else:
        period_returns = df['Log_Return'].rolling(window=day).sum().dropna()
    
    # VaR yüzdesini hesapla
    var_percentile = np.percentile(period_returns, (1 - confidence_level) * 100)
    
    var_value = portfolio_value * abs(var_percentile)
    
    return var_value

## VaR vs Historical VaR

In [15]:
# --- PARAMETRELER ---
portfolio_value = 100
confidence = 0.50
days = 10           

# --- 1. HULL LINEAR MODEL (PARAMETRİK) ---
var_param, _ = calculate_parametric_var(data, portfolio_value, confidence, days)

# --- 2. HULL HISTORICAL SIMULATION ---
var_hist = calculate_historical_var(data, portfolio_value, confidence, days)

print("-" * 40)
print(f"RİSK KARŞILAŞTIRMASI (Güven: %{confidence*100:.0f})")
print("-" * 40)
print(f"Parametrik VaR (Normal Dağılım, {days} gün):  ${var_param:.2f}")
print(f"Tarihsel VaR (Gerçek Veri, {days} gün):       ${var_hist:.2f}")
print("-" * 40)

fark = var_hist - var_param
if fark > 0:
    print(f"SONUÇ: Tarihsel VaR daha yüksek çıktı (Piyasa 'Normal'den daha riskli!)")
    print(f"Fark: ${fark:.2f} (Fat Tail Etkisi)")
else:
    print(f"SONUÇ: Parametrik VaR daha yüksek çıktı (Son {days} gün sakin geçmiş).")

plot_dashboard(data, symbol)

----------------------------------------
RİSK KARŞILAŞTIRMASI (Güven: %50)
----------------------------------------
Parametrik VaR (Normal Dağılım, 10 gün):  $0.00
Tarihsel VaR (Gerçek Veri, 10 gün):       $0.53
----------------------------------------
SONUÇ: Tarihsel VaR daha yüksek çıktı (Piyasa 'Normal'den daha riskli!)
Fark: $0.53 (Fat Tail Etkisi)


## VaR of Portfolio

In [27]:
def parametric_var_portfolio(df_portfolio, weights, portfolio_value=100000, confidence_level=0.95, day=1):
    """
    Portföy için Hull'un 'Linear Model'ini (Parametrik VaR) kullanarak riski hesaplar.
    """
    weights = np.array(weights)
    
    portfolio_volatility = portfolio_performance_with_data(df_portfolio, weights)[1]
    
    daily_volatility = portfolio_volatility / np.sqrt(252)

    z_score = norm.ppf(confidence_level)

    var_value = portfolio_value * daily_volatility * z_score * np.sqrt(day)
    
    return var_value, portfolio_volatility

symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA"]
weights = [0.15, 0.15, 0.15, 0.15, 0.2, 0.1, 0.1]  # Example Weights

trust = 0.99
value = 1000
df_portfolio = get_portfolio_history(symbols, period='1y')

print("-" * 40)
var_portfolio, port_volatility = parametric_var_portfolio(df_portfolio, weights, portfolio_value=value, confidence_level=trust, day=1)
print(f"Portföy Parametrik VaR ({trust*100:.0f}% güven seviyesi) : ${var_portfolio:.2f}")
print(f"Portföy Volatilitesi (Risk): %{port_volatility*100:.2f}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

----------------------------------------
Portföy Parametrik VaR (99% güven seviyesi) : $44.04
Portföy Volatilitesi (Risk): %30.05


## Some fun with Magnificent 7 stocks
How can we find the portfolio with minimum risk?

In [17]:
# Biraz eğlenceli optimizasyon yapalım
# --- OPTİMİZASYON DÖNGÜSÜ ---
num_simulations = 5000
min_var = float('inf')
best_weights = []

print(f"\n🔄 {num_simulations} farklı senaryo deneniyor...")

for _ in range(num_simulations):
    w = np.random.rand(7)
    w /= w.sum()
    
    current_var = parametric_var_portfolio(df_portfolio, w, value, trust, day=1)[0]
    
    # 3. En Düşüğü Kaydet (King of the Hill algoritması)
    if current_var < min_var:
        min_var = current_var
        best_weights = w

print("-" * 40)
print(f"🏆 MİNİMUM RİSK PORTFÖYÜ (VaR: ${min_var:.2f})")
print("-" * 40)
for sym, weight in zip(symbols, best_weights):
    print(f"{sym}: %{weight*100:.2f}")
print("-" * 40)


🔄 5000 farklı senaryo deneniyor...
----------------------------------------
🏆 MİNİMUM RİSK PORTFÖYÜ (VaR: $35.64)
----------------------------------------
AAPL: %23.87
MSFT: %41.17
GOOGL: %11.01
AMZN: %16.94
NVDA: %1.95
META: %3.93
TSLA: %1.13
----------------------------------------
----------------------------------------
🏆 MİNİMUM RİSK PORTFÖYÜ (VaR: $35.64)
----------------------------------------
AAPL: %23.87
MSFT: %41.17
GOOGL: %11.01
AMZN: %16.94
NVDA: %1.95
META: %3.93
TSLA: %1.13
----------------------------------------


## Historical VaR of Portfolio

In [33]:
def historical_var_portfolio(df_portfolio, weights, portfolio_value=100000, confidence_level=0.95, day=1):
    """
    Portföy için 'Historical Simulation' (Gerçek Veri) VaR hesabı.
    Ağırlıklandırılmış geçmiş getirileri kullanır.
    """
    weights = np.array(weights)
    
    portfolio_historical_returns = df_portfolio.dot(weights)
    portfolio_historical_returns = portfolio_historical_returns.dropna()
    # 2. ADIM: Rolling Sum (Doğru Historical Yöntem)
    if day > 1:
        portfolio_historical_returns = portfolio_historical_returns.rolling(window=day).sum().dropna()
    
    # 3. ADIM: Percentile
    var_percentile = np.percentile(portfolio_historical_returns, (1 - confidence_level) * 100)
    
    return portfolio_value * abs(var_percentile)


symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA"]
weights = [0.15, 0.15, 0.15, 0.15, 0.2, 0.1, 0.1]  # Example Weights
trust = 0.99
value = 1000

var_hist_portfolio = historical_var_portfolio(df_portfolio, weights, portfolio_value=value, confidence_level=trust, day=1)

In [34]:
print("-" * 40)
print(f"Portföy Tarihsel VaR ({trust*100:.0f}% güven seviyesi) : ${var_hist_portfolio:.2f}")

----------------------------------------
Portföy Tarihsel VaR (99% güven seviyesi) : $52.64


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Senin fonksiyonunun içindeki hesaplamayı dışarı alıp görselleştiriyoruz
# 1. Sanal Portföyü Hesapla
portfolio_hist_ret = df_portfolio.dot(np.array(weights)).dropna()

# 2. %1'lik Sınırı Bul (Percentile)
cutoff = np.percentile(portfolio_hist_ret, (1 - trust) * 100)

# --- GRAFİK ---
plt.figure(figsize=(12, 6))

# Histogram (Dağılım)
sns.histplot(portfolio_hist_ret, bins=50, kde=True, color='skyblue', label='Günlük Getiriler')

# VaR Çizgisi (Kırmızı Çizgi)
plt.axvline(x=cutoff, color='red', linestyle='--', linewidth=3, label=f'VaR (%99): {cutoff:.2%}')

# Süsleme
plt.title('Magnificent 7 Portföyü - Tarihsel Getiri Dağılımı ve Risk Sınırı', fontsize=14)
plt.xlabel('Günlük Getiri')
plt.ylabel('Frekans (Gün Sayısı)')
plt.legend()
plt.grid(True, alpha=0.3)

# Sol taraftaki "Kuyruk" (Tail) bölgesini boya
plt.axvspan(portfolio_hist_ret.min(), cutoff, color='red', alpha=0.2)

plt.show()

print(f"📊 Hesaplanan Tarihsel VaR Tutarı: ${var_hist_portfolio:.2f}")
print(f"📉 Bu şu demek: Geçmişe bakıldığında, 100 günün 1'inde {cutoff:.2%} veya daha fazla kayıp yaşanmış.")